# This notebook explores the Cooley Tukey algorithm for FFT

In [1]:
import numpy as np

References: https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/

# Dicrete Fourier Transform
let $x_n$ denote the coordinate of configuration spaces and let $X_n$ be the frequency space.  
Then discrete forward Fourier Transform is given by
$$X_k=\sum_{n=0}^{N-1}x_n\cdot e^{-i~2\pi kn/N}$$
and inverse by
$$x_n=\frac{1}{N}\sum_{k=0}^{N-1}X_k e^{-i~2\pi kn/N}$$

Note that obtaining $X$ Is nothing more than taking the dot product of $x$ and some matrix M so that
$$\vec{X}=M\cdot \vec{x}$$
where M is of the form
$$M_{kn}=e^{-i~2\pi kn/N}$$

In [137]:
def DFT_slow(x):
    """Compute the discrete Fourier Transform of the 1D array x"""
    x=np.asarray(x,dtype=float)
    N=x.shape[0]
    n=np.arange(N)
    k=n.reshape((N,1))
    M = np.exp(-2j * np.pi * k * n / N)
    
    return np.dot(M,x)

make sure it coincides with built in function numpy.fft.fft()

In [138]:
x=np.random.random(123)
#print(DFT_slow(y))
np.allclose(DFT_slow(x), np.fft.fft(x))

True

measure time difference between fft and my dft

In [139]:
%timeit DFT_slow(x)
%timeit np.fft.fft(x)

1000 loops, best of 3: 1.08 ms per loop
The slowest run took 13.25 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 8.92 µs per loop


To reduce computation time, I exploit the symmetry.  
Consider

\begin{align}
X_{N+k}&=\sum_{n=0}^{N-1}x_ne^{-i~2\pi (N+k)n/N}\\
&=\sum_{n=0}^{N-1}x_ne^{-i~2\pi n}e^{-i~2\pi kn/N}\\
&=\sum_{n=0}^{N-1}x_ne^{-i~2\pi kn/N}
\end{align}

Thus
$$X_{N+k}=X_N$$

Then it's obvious that
$$X_{k+hN}=X_N$$ for any integer h.

Cooley and Tucky showed that the DFT can be 'divided' as follows

